In [ ]:
f_func <- "./functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))

In [ ]:
samples_metadata <- read.csv(file = "../../output/samples_metadata.csv") %>% as.data.table()

In [ ]:
host_nonMT_IS_meta <- read.csv(file = "../../output/PP_figures/host_new_nonMT_IS_meta.csv") %>% 
    filter(origin != "controls") %>% 
    filter(type != "ds-wcDNA") %>%    
    mutate(TLEN = abs(TLEN)) %>% 
    filter(TLEN >= -500 & TLEN <= 500) %>% 
    mutate(read = substr(side,1,2)) 

In [ ]:
host_MT_IS_meta <- read.csv(file = "../../output/PP_figures/host_new_MT_IS_meta.csv") %>% 
    filter(origin != "controls") %>% 
    filter(type != "ds-wcDNA") %>%    
    mutate(TLEN = abs(TLEN)) %>% 
    filter(TLEN >= -500 & TLEN <= 500) %>% 
    mutate(read = substr(side,1,2)) 

In [ ]:
host_ALL_IS_meta <- rbind(read.csv(file = "../../output/PP_figures/host_new_MT_IS_meta.csv"), 
                        read.csv(file = "../../output/PP_figures/host_new_nonMT_IS_meta.csv"))%>% 
    filter(origin != "controls") %>% 
    filter(type != "ds-wcDNA") %>%    
    mutate(TLEN = abs(TLEN)) %>% 
    filter(TLEN >= -500 & TLEN <= 500) %>% 
    mutate(read = substr(side,1,2)) 

In [ ]:
Afum_IS_meta <- read.csv(file = "../../output/PP_figures/746128_IS_meta.csv") %>% 
    filter(mapGenome == "AfumigatusAf293") %>% 
    filter(origin != "controls") %>% 
    filter(type != "ds-wcDNA") %>%    
    mutate(TLEN = abs(TLEN)) %>% 
    filter(TLEN >= -500 & TLEN <= 500) %>% 
    mutate(read = substr(side,1,2)) 

In [ ]:
host_nonMT <- host_nonMT_IS_meta %>% 
    group_by(sample_id, TLEN, sample_type, sample, type, patient, origin) %>% 
    summarise(sCount = sum(Count, na.rm = TRUE), .groups = "keep") %>%     
    group_by(sample_id, sample_type, sample, type, patient, origin) %>% 
    mutate(nsCount = sCount/sum(sCount, na.rm = TRUE)*100) 

host_nonMT_summarized <- host_nonMT %>% 
    group_by(TLEN, sample_type, type) %>% 
    summarise(mnsCount = mean(nsCount, na.rm = TRUE), .groups = "keep")

In [ ]:
host_nonMT_endX_TLEN <- host_nonMT_IS_meta %>% 
    mutate(EndX = substr(EndMotif,1,1)) %>% 
    group_by(sample_id, EndX, TLEN, sample_type, sample, type, read) %>% 
    summarise(sCount = sum(Count, na.rm = TRUE), .groups = "keep") %>%    
    group_by(sample_id, TLEN, sample_type, sample, type, read) %>% 
    mutate(nsCount = sCount/sum(sCount, na.rm = TRUE)*100) 

host_nonMT_endX_TLEN_summarized <- host_nonMT_endX_TLEN %>% 
    group_by(TLEN, EndX, sample_type, sample, type, read) %>% 
    mutate(mnsCount = mean(nsCount))

In [ ]:
host_nonMT_endX <- host_nonMT_IS_meta %>% 
    mutate(EndX = substr(EndMotif,1,1)) %>% 
    group_by(sample_id, origin, EndX, sample_type, sample, type, read) %>% 
    summarise(sCount = sum(Count, na.rm = TRUE), .groups = "keep") %>%
    group_by(sample_id, origin, sample_type, sample, type, read) %>% 
    mutate(nsCount = sCount/sum(sCount, na.rm = TRUE)*100)

host_nonMT_endXX <- host_nonMT_IS_meta %>% 
    mutate(EndX = substr(EndMotif,1,1)) %>% 
    mutate(EndXX = substr(EndMotif,1,2)) %>% 
    group_by(sample_id, origin, EndX, EndXX, sample_type, sample, type, read) %>% 
    summarise(sCount = sum(Count, na.rm = TRUE), .groups = "keep") %>%
    group_by(sample_id, origin, sample_type, sample, type, read) %>% 
    mutate(nsCount = sCount/sum(sCount, na.rm = TRUE)*100)

In [ ]:
Afum_endX <- Afum_IS_meta %>% 
    mutate(EndX = substr(EndMotif,1,1)) %>% 
    group_by(sample_id, origin, EndX, sample_type, sample, type, read) %>% 
    summarise(sCount = sum(Count, na.rm = TRUE), .groups = "keep") %>%
    group_by(sample_id, origin, sample_type, sample, type, read) %>% 
    mutate(nsCount = sCount/sum(sCount, na.rm = TRUE)*100)

Afum_endXX <- Afum_IS_meta %>% 
    mutate(EndX = substr(EndMotif,1,1)) %>% 
    mutate(EndXX = substr(EndMotif,1,2)) %>% 
    group_by(sample_id, origin, EndX, EndXX, sample_type, sample, type, read) %>% 
    summarise(sCount = sum(Count, na.rm = TRUE), .groups = "keep") %>%
    group_by(sample_id, origin, sample_type, sample, type, read) %>% 
    mutate(nsCount = sCount/sum(sCount, na.rm = TRUE)*100)

In [ ]:
#other normalizaiton, per bin, for clustering
host_nonMT_endXX_binned <- host_nonMT_IS_meta %>% 
    mutate(EndXX = substr(EndMotif,1,2)) %>%    

    group_by(sample_id, origin, EndXX, sample_type, sample, type, read) %>%
    mutate(TLEN_bin = cut(TLEN, breaks = seq(35, 500, by = 1), include.lowest = TRUE)) %>%

    group_by(sample_id, origin, EndXX, sample_type, sample, type, read, TLEN_bin) %>%
    summarise(binCount = sum(Count, na.rm = TRUE), .groups = "keep") %>%

    group_by(sample_id, origin, sample_type, sample, type, read, TLEN_bin) %>% 
    mutate(binCountNorm = binCount/sum(binCount, na.rm = TRUE)*100) %>% 
    
    group_by(sample_id, origin, EndXX, sample_type, sample, type, read) %>% 
    summarise(nsCount = mean(binCountNorm), .groups = "keep") 

In [ ]:
plasma_sizeSelection <- read.table("../../output/sizeSelection/plasma_sizeSelection_output.txt", 
                                   header = TRUE, sep = "\t") 
BAL_sizeSelection <- read.table("../../output/sizeSelection/BAL_sizeSelection_output.txt", 
                                header = TRUE, sep = "\t") 

In [ ]:
plasma_sizeSelection_meta <- merge(x = samples_metadata, y = plasma_sizeSelection, by.x = "sample_id", by.y = "sample_id")  %>%
    mutate(taxID_number_reads = ifelse(taxID_number_reads == QC_number_reads, yes = 0, no = taxID_number_reads)) %>% 
    mutate(taxName = case_when(
        taxID == 1131492 ~ "Aspergillaceae", 
        taxID == 5052 ~ "Aspergillus",
        taxID == 746128 ~ "Aspergillus fumigatus" # If there are other taxID values, keep them as is or change the default
    )) %>% 
    filter(type != "ds-wcDNA") %>% filter(origin != "control") %>% 
    mutate(new_Filter = gsub(Filter, pattern = "to", replacement = "-")) %>% 
    mutate(new_Filter = factor(new_Filter, levels = c('35-50','35-75','35-100','35-125','35-150','35-1000','noFilter'))) 

BAL_sizeSelection_meta <- merge(x = samples_metadata, y = BAL_sizeSelection, by.x = "sample_id", by.y = "sample_id")  %>% 
    mutate(taxID_number_reads = ifelse(taxID_number_reads == QC_number_reads, yes = 0, no = taxID_number_reads)) %>% 
    mutate(taxName = case_when(
        taxID == 1131492 ~ "Aspergillaceae", 
        taxID == 5052 ~ "Aspergillus",
        taxID == 746128 ~ "Aspergillus fumigatus" # If there are other taxID values, keep them as is or change the default
    )) %>% 
    filter(type != "ds-wcDNA") %>% filter(origin != "control") %>% 
    mutate(new_Filter = gsub(Filter, pattern = "to", replacement = "-")) %>% 
    mutate(new_Filter = factor(new_Filter, levels = c('35-50','35-75','35-100','35-125','35-150','35-1000','noFilter'))) 

In [ ]:
QCreads_endX <- read.table("../../output/PP_figures/QCreads_EndMotifs.csv", 
           header = TRUE, sep = ",") %>% 
    mutate(EndX = substr(EndMotif,1,1)) %>% 
    group_by(sample_id, sample_type, sample, type, patient, origin, read, EndX) %>% 
    summarise(sCount = sum(Count, na.rm = TRUE), .groups = "keep") 

QCreads_endXX <- read.table("../../output/PP_figures/QCreads_EndMotifs.csv", 
           header = TRUE, sep = ",") %>% 
    mutate(EndX = substr(EndMotif,1,1)) %>% 
    mutate(EndXX = substr(EndMotif,1,2)) %>% 
    group_by(sample_id, sample_type, sample, type, patient, origin, read, EndX, EndXX) %>% 
    summarise(sCount = sum(Count, na.rm = TRUE), .groups = "keep") 

host_nonMT_endX <- host_nonMT_IS_meta %>% 
    mutate(EndX = substr(EndMotif,1,1)) %>% 
    group_by(sample_id, origin, EndX, sample_type, sample, type, read) %>% 
    summarise(sCount = sum(Count, na.rm = TRUE), .groups = "keep") %>%    
    group_by(sample_id, origin, sample_type, sample, type, read) %>% 
    mutate(nsCount = sCount/sum(sCount, na.rm = TRUE)*100)  

host_nonMT_endX_33ntSplit <- host_nonMT_IS_meta %>% 
    mutate(TLEN_bin = cut(TLEN, breaks = seq(0, 500, by = 100), include.lowest = TRUE)) %>% 
    mutate(EndX = substr(EndMotif,1,1)) %>% 
    group_by(sample_id, origin, TLEN_bin, EndX, sample_type, sample, type, read) %>% 
    summarise(sCount = sum(Count, na.rm = TRUE), .groups = "keep") %>%    
    group_by(sample_id, origin, TLEN_bin, sample_type, sample, type, read) %>% 
    mutate(nsCount = sCount/sum(sCount, na.rm = TRUE)*100) 

host_nonMT_endXX <- host_nonMT_IS_meta %>% 
    mutate(EndXX = substr(EndMotif,1,2)) %>% 

    group_by(sample_id, origin, EndXX, sample_type, sample, type, read) %>% 
    summarise(sCount = sum(Count, na.rm = TRUE), .groups = "keep") %>%    

    group_by(sample_id, origin, sample_type, sample, type, read) %>% 
    mutate(nsCount = sCount/sum(sCount, na.rm = TRUE)*100)  

host_nonMT_endXXX <- host_nonMT_IS_meta %>%  
    mutate(EndX = substr(EndMotif,1,3)) %>% 
    group_by(sample_id, origin, EndX, sample_type, sample, type, read) %>% 
    summarise(sCount = sum(Count, na.rm = TRUE), .groups = "keep") %>%    
    group_by(sample_id, origin, sample_type, sample, type, read) %>% 
    mutate(nsCount = sCount/sum(sCount, na.rm = TRUE)*100)  